In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd

In [4]:
from ecephys.graham import paths, channel_groups
from ecephys.utils.utils import store_df_h5
from ecephys.sglx_utils import load_timeseries
from ecephys.scoring import load_visbrain_hypnogram
from ecephys.signal.ripples import make_speed_vector, apply_ripple_filter, generalized_Kay_ripple_detector, compute_ripple_features
from ecephys.plot import lazy_ripple_explorer

In [27]:
condition = "pre-SR"
subject = "Valentino"

## Load the data

In [28]:
chans = channel_groups.CA1[subject]

In [29]:
bin_path = Path(paths.lfp_bin[condition][subject])
(time, lfps, sampling_frequency) = load_timeseries(bin_path, chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000000


## Load hypnogram and select only NREM epochs

In [30]:
#hypnogram_path = Path(paths.hypnograms[condition][subject])
#hypnogram = load_visbrain_hypnogram(hypnogram_path)

In [31]:
#detect_states = ['Wake', 'N1', 'N2', 'REM']
#speed = make_speed_vector(hypnogram, detect_states, time)
speed = np.zeros_like(time)

## Apply ripple bandpass filter

In [32]:
filtered_lfps = apply_ripple_filter(lfps, sampling_frequency)

## Detect Ripples

### Kay method
Detection of SWRs was prerequisite for all data analyzed in this study, and was performed only when at least three CA1 cell layer recordings were available. Offline, a multisite average approach was used to detect SWRs57. Specifically, LFPs from all available CA1 cell layer tetrodes were filtered between 150–250 Hz, then squared and summed across tetrodes. This sum was smoothed with a Gaussian kernel (σ = 4 ms) and the square root of the smoothed sum was analyzed. SWRs were detected when the signal exceeded 2 s.d. of the recording epoch mean for at least 15 ms. SWR periods were then defined as the periods, containing the times of threshold crossing, in which the power trace exceeded the mean. SWR onset was defined as the start of a SWR period. Detection of SWRs was performed only when subjects' head speed was <4 cm/s. For SWR-triggered spike raster plots and PSTH plots, a 0.5 s exclusion period was imposed to isolate SWRs occurring only after non-SWR periods; otherwise, analyses of SWRs included all detected SWRs.

### Karlsson method
Sharp waves ripple events (SWRs) were identified based on peaks in the local field potential (LFP) recorded from one channel from each tetrode in the CA3 and CA1 cell layers. The raw LFP data were band-pass filtered between 150–250 Hz and the SWR envelope was determined using a Hilbert transform. The envelope was smoothed with a Gaussian (4 ms stdev). We initially identified SWR events as sets of times when the smoothed envelope stayed above 3 standard deviations of the mean for at least 15 ms on at least one tetrode. We defined the entire SWR as including times immediately before and after that threshold crossing event during which the envelope exceeded the mean. Overlapping SWRs were combined across tetrodes, so many events extended beyond the SWR seen on a single tetrode.

### Eschenko-like method
SPW-Rs were detected by means of an automatic thresholding algorithm (Siapas and Wilson 1998; Csicsvari et al. 1999), illustrated in Figure 1. The hippocampal LFP signal was first filtered (150–250 Hz) and the root mean square (RMS) was calculated at every 5 msec (in a moving 10-msec window). The threshold for ripple peak detection was set to 4 SDs above the mean RMS signal. The beginning and end of a ripple were marked at points at which the RMS signal dropped below 2 SD provided that these two points were separated by 25 msec; these time points were used to estimate ripple duration. For every marked ripple, the troughs of the ripples were detected as the minima of the filtered LFP signal, and the deepest trough was marked as the time point representing the respective SPW-R event. The magnitude of the ripple was determined by integrating the RMS signal over the duration of each ripple (King et al. 1999). Ripples were detected during SWS periods only. The ripple amplitude was estimated as a voltage difference between maximum and minimum peaks of the filtered CA1 signal during the ripple event. Ripple density was defined as the number of ripples/second. Figure 1A shows the raw data from a hippocampal electrode. Ripple parameters and how they were measured are illustrated in Figure 1B.

In [33]:
detection_zscore_threshold=4.0
boundary_zscore_threshold=1.0
minimum_duration=0.025
method = 'Kay-like'
ripple_times = generalized_Kay_ripple_detector(time, filtered_lfps, speed, sampling_frequency, detection_zscore_threshold=detection_zscore_threshold, boundary_zscore_threshold=boundary_zscore_threshold, minimum_duration=minimum_duration)

## Collect ripple statistics

In [34]:
ripple_features = compute_ripple_features(time, filtered_lfps, ripple_times, sampling_frequency, 'Kay')

## Export results

In [35]:
metadata = dict(
    chans=chans,
    detect_states=["Wake", "N1", "N2", "REM"],
    detection_zscore_threshold=4.0,
    boundary_zscore_threshold=1.0,
    minimum_duration=0.025,
    method="Kay-like",
)
ripple_path = Path(paths.ripples[condition][subject])
store_df_h5(ripple_path, ripple_features, **metadata)

## Explore results

In [36]:
lazy_ripple_explorer(ripple_features, metadata, subject, condition)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='window_length', max=2.0, min=0.25, step=0.25), IntSl…